# Entrega extra evaluación: análisis_ventas_en_python
Autor: **Manal Zahi**

En este notebook cargo los datos de ventas, clientes y productos, los limpio siguiendo las reglas del enunciado y genero tres CSV finales listos para análisis.
Voy dejando comentarios y pequeñas comprobaciones para ir viendo que todo tiene sentido.


In [ ]:
import pandas as pd
import numpy as np

# En el mismo directorio del notebook deben estar ventas.csv, clientes.csv y productos.csv
ventas_path = "ventas.csv"
clientes_path = "clientes.csv"
productos_path = "productos.csv"

# 1) Carga de datos
df_ventas = pd.read_csv(ventas_path)
df_clientes = pd.read_csv(clientes_path)
df_productos = pd.read_csv(productos_path)

# Echo un vistazo rápido a las primeras filas y a los tipos
display(df_ventas.head())
display(df_clientes.head())
display(df_productos.head())

print("\nTipos originales df_ventas:\n", df_ventas.dtypes)
print("\nTipos originales df_clientes:\n", df_clientes.dtypes)
print("\nTipos originales df_productos:\n", df_productos.dtypes)


In [ ]:
# 1) Ajuste de tipos: fechas como datetime y algunos campos numéricos
df_ventas["fecha_venta"] = pd.to_datetime(df_ventas["fecha_venta"], errors="coerce")
df_clientes["fecha_registro"] = pd.to_datetime(df_clientes["fecha_registro"], errors="coerce")
df_productos["fecha_alta"] = pd.to_datetime(df_productos["fecha_alta"], errors="coerce")

# cliente_id en df_ventas como entero nullable (para poder tener NaN antes de limpiar)
df_ventas["cliente_id"] = df_ventas["cliente_id"].astype("Int64")

# edad y activo en df_clientes
df_clientes["edad"] = df_clientes["edad"].astype("float")
df_clientes["activo"] = df_clientes["activo"].astype("Int64")

# stock en productos como float (ya viene así) y luego lo dejaremos en entero
df_productos["stock"] = df_productos["stock"].astype("float")

print("\nTipos corregidos df_ventas:\n", df_ventas.dtypes)
print("\nTipos corregidos df_clientes:\n", df_clientes.dtypes)
print("\nTipos corregidos df_productos:\n", df_productos.dtypes)


In [ ]:
# 2) Detección y eliminación de duplicados
# df_ventas: misma venta_id, cliente_id, producto_id, fecha_venta
dups_ventas_mask = df_ventas.duplicated(
    subset=["venta_id", "cliente_id", "producto_id", "fecha_venta"]
)
num_dups_ventas = dups_ventas_mask.sum()
print("Duplicados en df_ventas según clave compuesta:", num_dups_ventas)

df_ventas_sin_duplicados = df_ventas.drop_duplicates(
    subset=["venta_id", "cliente_id", "producto_id", "fecha_venta"], keep="first"
)

# df_clientes: mismos nombre, apellido, fecha_registro
dups_clientes_mask = df_clientes.duplicated(
    subset=["nombre", "apellido", "fecha_registro"]
)
num_dups_clientes = dups_clientes_mask.sum()
print("Duplicados en df_clientes según nombre+apellido+fecha_registro:", num_dups_clientes)

df_clientes_sin_duplicados = df_clientes.drop_duplicates(
    subset=["nombre", "apellido", "fecha_registro"], keep="first"
)

# A partir de aquí trabajo ya con las versiones sin duplicados
df_ventas_clean = df_ventas_sin_duplicados.copy()
df_clientes_clean = df_clientes_sin_duplicados.copy()
df_productos_clean = df_productos.copy()


In [ ]:
# 3) Rellenar valores faltantes
# --- df_ventas ---
# cantidad vacía -> 0
df_ventas_clean["cantidad"] = df_ventas_clean["cantidad"].fillna(0)

# precio vacío -> media del precio por producto_id
precio_medio_por_producto = df_ventas_clean.groupby("producto_id")["precio"].transform("mean")
df_ventas_clean["precio"] = df_ventas_clean["precio"].fillna(precio_medio_por_producto)

# categoria vacía -> "SinCategoria"
df_ventas_clean["categoria"] = df_ventas_clean["categoria"].fillna("SinCategoria")

# descuento vacío -> según tabla de máximos por categoría
tabla_descuentos = {
    "Electrónica": 0.25,
    "Hogar": 0.10,
    "Deporte": 0.20,
    "SinCategoria": 0.00
}
mask_desc_na = df_ventas_clean["descuento"].isna()
df_ventas_clean.loc[mask_desc_na, "descuento"] = df_ventas_clean.loc[mask_desc_na, "categoria"].map(tabla_descuentos)

# Para cualquier fila con categoria "SinCategoria", fuerzo descuento = 0.0
df_ventas_clean.loc[df_ventas_clean["categoria"] == "SinCategoria", "descuento"] = 0.0

# fecha_venta vacía -> fecha mínima del propio df
min_fecha_venta = df_ventas_clean["fecha_venta"].min()
df_ventas_clean["fecha_venta"] = df_ventas_clean["fecha_venta"].fillna(min_fecha_venta)

# cliente_id vacío -> eliminar fila completa
antes_drop = df_ventas_clean.shape[0]
df_ventas_clean = df_ventas_clean.dropna(subset=["cliente_id"])
despues_drop = df_ventas_clean.shape[0]
print("Filas eliminadas en ventas por cliente_id vacío:", antes_drop - despues_drop)

# Ajustes finales de tipo
df_ventas_clean["cliente_id"] = df_ventas_clean["cliente_id"].astype("Int64")
df_ventas_clean["cantidad"] = df_ventas_clean["cantidad"].astype("Int64")

# --- df_clientes ---
# edad vacía -> media por ciudad, si falta ciudad -> media global, y redondeo
media_global_edad = df_clientes_clean["edad"].mean()
media_edad_por_ciudad = df_clientes_clean.groupby("ciudad")["edad"].transform("mean")
edad_relleno = media_edad_por_ciudad.fillna(media_global_edad)
mask_edad_na = df_clientes_clean["edad"].isna()
df_clientes_clean.loc[mask_edad_na, "edad"] = edad_relleno[mask_edad_na]
df_clientes_clean["edad"] = df_clientes_clean["edad"].round().astype("Int64")

# fecha_registro vacía -> fecha mínima
min_fecha_reg = df_clientes_clean["fecha_registro"].min()
df_clientes_clean["fecha_registro"] = df_clientes_clean["fecha_registro"].fillna(min_fecha_reg)

# email vacío -> "desconocido@example.com"
df_clientes_clean["email"] = df_clientes_clean["email"].fillna("desconocido@example.com")

# activo vacío -> 0
df_clientes_clean["activo"] = df_clientes_clean["activo"].fillna(0).astype("Int64")

# --- df_productos ---
# stock vacío -> 0
df_productos_clean["stock"] = df_productos_clean["stock"].fillna(0)

# precio vacío -> coste * 1.5 (por si se diera el caso)
mask_precio_na = df_productos_clean["precio"].isna()
df_productos_clean.loc[mask_precio_na, "precio"] = df_productos_clean.loc[mask_precio_na, "coste"] * 1.5

# categoria vacía -> "SinCategoria"
df_productos_clean["categoria"] = df_productos_clean["categoria"].fillna("SinCategoria")

# Tipos finales de productos
df_productos_clean["stock"] = df_productos_clean["stock"].astype("Int64")

print("\nComprobación de nulos tras limpieza (ventas):\n", df_ventas_clean.isna().sum())
print("\nComprobación de nulos tras limpieza (clientes):\n", df_clientes_clean.isna().sum())
print("\nComprobación de nulos tras limpieza (productos):\n", df_productos_clean.isna().sum())


In [ ]:
# 4) Filtrado de filas
# df_ventas_filtrado: cantidad>0, descuento>0, categoria!=SinCategoria, fecha_venta>=2024-01-10
df_ventas_filtrado = df_ventas_clean[
    (df_ventas_clean["cantidad"] > 0) &
    (df_ventas_clean["descuento"] > 0) &
    (df_ventas_clean["categoria"] != "SinCategoria") &
    (df_ventas_clean["fecha_venta"] >= pd.to_datetime("2024-01-10"))
].copy()
print("Número de filas en df_ventas_filtrado:", df_ventas_filtrado.shape[0])

# df_clientes_activos: activo==1 y edad>=30
df_clientes_activos = df_clientes_clean[
    (df_clientes_clean["activo"] == 1) &
    (df_clientes_clean["edad"] >= 30)
].copy()
print("Número de filas en df_clientes_activos:", df_clientes_activos.shape[0])


In [ ]:
# 5) Columnas derivadas
# df_ventas_new con importe_bruto e importe_neto
df_ventas_new = df_ventas_clean.copy()
df_ventas_new["importe_bruto"] = df_ventas_new["cantidad"] * df_ventas_new["precio"]
df_ventas_new["importe_neto"] = df_ventas_new["importe_bruto"] * (1 - df_ventas_new["descuento"])

# df_productos_new con margen y porcentaje_margen
df_productos_new = df_productos_clean.copy()
df_productos_new["margen"] = df_productos_new["precio"] - df_productos_new["coste"]
df_productos_new["porcentaje_margen"] = df_productos_new["margen"] / df_productos_new["precio"]

# df_clientes_new con es_mayor_edad
df_clientes_new = df_clientes_clean.copy()
df_clientes_new["es_mayor_edad"] = df_clientes_new["edad"] >= 18

# Mostrar primeras filas de df_ventas_filtrado con las nuevas columnas
ventas_filtrado_new = df_ventas_new.merge(
    df_ventas_filtrado[["venta_id"]], on="venta_id", how="inner"
)
display(ventas_filtrado_new.head())

# Calcular máximo importe_neto y mínimo porcentaje_margen
max_importe_neto = df_ventas_new["importe_neto"].max()
min_porcentaje_margen = df_productos_new["porcentaje_margen"].min()
print("Máximo importe_neto:", max_importe_neto)
print("Mínimo porcentaje_margen:", min_porcentaje_margen)


In [ ]:
# 6) Guardar los DataFrames resultado en CSV
ventas_limpias_path = "ventas_limpias.csv"
clientes_limpios_path = "clientes_limpios.csv"      # df_productos_new
productos_limpios_path = "productos_limpios.csv"    # df_clientes_new

df_ventas_new.to_csv(ventas_limpias_path, index=False)
df_productos_new.to_csv(clientes_limpios_path, index=False)
df_clientes_new.to_csv(productos_limpios_path, index=False)

print("CSV generados:")
print("-", ventas_limpias_path)
print("-", clientes_limpios_path)
print("-", productos_limpios_path)
